# 機械学習

## 目次

- テストデータと訓練データ
- オーバーフィッティングとアンダーフィッティング
- ハイパーパラメータの調整
  - 交差検証
  - グリッドサーチ
- モデルの評価

## Section3 学習と評価の方法

In [ ]:
# # Googleドライブのマウント（Colab使いのみ）

# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/dlc/week1

In [ ]:
# 図表が使えるようにする

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### 3.1 テストデータと訓練データ

機械学習モデルを学習させる場合、手元にあるデータを **訓練データ (Train data)** と、 **テストデータ (Test data)** に予め分割します。これをホールドアウト法と言います。

訓練データはモデルが学習するため、テストデータはモデルが「どれくらい正しく予測できているか」を評価するためのデータです。

学習する際は、テストデータが全く見えない状態で行う必要が有ります。逆に、テストデータの中にあるべき変数が訓練データに洩れてしまっていることを **リーク (Data leakage)** と言います。

In [ ]:
# データの分割

from sklearn.model_selection import train_test_split
import pandas as pd

# 訓練データのロード
data = pd.read_csv('./data/train.csv')
feature = data.drop(['Survived'], axis=1)
target = data['Survived']

# テストデータ30%で分割する
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3)

前処理をする際も、訓練データとテストデータは分けるようにしましょう。

In [ ]:
# 前処理をする関数
## train : pd.DataFrame 
## test : pd.DataFrame
def preProcessing(train, test):
    
    # PassengerId, Name, Ticket, Cabinを削除する
    train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    
    # Ageの欠損値を中央値で埋める
    fill_age = train['Age'].median()
    train['Age'] = train['Age'].fillna(fill_age)
    test['Age'] = test['Age'].fillna(fill_age)

    # Embarkedの欠損値を最頻値"S"で埋める
    fill_embarked = 'S'
    train['Embarked'] = train['Embarked'].fillna(fill_embarked)
    test['Embarked'] = test['Embarked'].fillna(fill_embarked)
    
    # 家族の数(Family_size)という特徴量を作る
    train['Family_size'] = train['SibSp'] + train['Parch'] + 1
    test['Family_size'] = test['SibSp'] + test['Parch'] + 1

    # SibSpとParchを削除する
    train = train.drop(['SibSp', 'Parch'], axis=1)
    test = test.drop(['SibSp', 'Parch'], axis=1)
    
    # カテゴリ変数をOne-Hotエンコーディング
    train = pd.get_dummies(train, columns=['Sex', 'Pclass', 'Embarked'])
    test = pd.get_dummies(test, columns=['Sex', 'Pclass', 'Embarked'])
    
    return train, test

In [ ]:
# 前処理

X_train1, X_test1 = preProcessing(X_train, X_test)
y_train1, y_test1 = y_train, y_test

学習させてみる

In [ ]:
# k-最近傍法 (knn)

from sklearn.neighbors import KNeighborsClassifier

knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train1, y_train1)

In [ ]:
# モデルの予測正解率を確認

y_pred = knn1.predict(X_test1)
print("Test set score: {:.2f}".format(knn1.score(X_test1, y_test1)))

### 3.2 オーバーフィッティングとアンダーフィッティング

モデルが訓練データに対してすら良い予測精度を示さないという状態は **アンダーフィッティング（適合不足/学習不足）** と呼ばれます。

一方で、訓練データに対しては良い予測精度を示すにも関わらず、テストデータに対してのパフォーマンスが良くない場合はモデルが訓練データに **オーバーフィッティング (過剰適合/過学習)** していると言います。

<img src="figures/sweetspot.png" width="60%">

画像の出典：https://qiita.com/kotamatsuoka/items/1ccb41ca278e400b6197

訓練データをもとにつくったモデルが、未知のデータに対して同じような予測精度を示すなら **汎化 (Generalize)** できているといいます。この汎化性能が機械学習に置いて最重要視すべき観点です。

一般的に、モデルを複雑にするほど訓練データに適合して行きます。適合不足にも過剰適合にもならない適度な複雑さの時に汎化能力が最大になるのでそこを目指しましょう。

アルゴリズムの動作を制御するためのパラーメータ値を **ハイパーパラメータ (Hyperparameter)** と言い、これを調整してモデルの複雑さなどを変更します。

In [ ]:
# k-最近傍法 (knn)のハイパーパラメータを変えてみる

from sklearn.neighbors import KNeighborsClassifier

knn2 = KNeighborsClassifier(n_neighbors=3)
knn2.fit(X_train1, y_train1)

In [ ]:
# モデルの予測正解率を確認

y_pred = knn2.predict(X_test1)
print("Test set score: {:.2f}".format(knn2.score(X_test1, y_test1)))

### 3.3 ハイパーパラメータの調整

前述のホールドアウト法ではデータセットを訓練用・テスト用に2分割しましたが、実際の開発時にはモデルの性能評価をより適切にするためにデータを3分割してモデルを評価することが一般的です。

| 名称 | 目的 |
| :-- | :-- |
| 訓練データ (Train data) | モデルの学習に利用 |
| 検証データ (Validation data) | ハイパーパラメータの調整に利用 |
| テストデータ (Test data) | モデルの最終性能評価に利用 |

In [ ]:
# テストデータ20%で分割する
X_train2, X_test2, y_train2, y_test2 = train_test_split(feature, target, test_size=0.2)

# 前処理
X_train2, X_test2 = preProcessing(X_train2, X_test2)
y_train2, y_test2 = y_train2, y_test2

# 検証データ20%(=80%x25%)で分割する
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train2, y_train2, test_size=0.25)

#### 3.3.1 交差検証

**交差検証 (Cross validation)** とは汎化性能を評価する統計的手法です。最も一般的な手法に **K-分割交差検証 (K-fold cross validation)** があります。

K-分割交差検証は、データを $k$ 個に分割し、モデルの訓練と評価を $k$ 回行います。得られた $k$ 個の評価値の平均値を最終的なモデルのスコアとして扱います。

データの量が少ない場合に有効です。

<img src="figures/cross_validation.png" width="100%">

In [ ]:
# 交差検証をしてみる

from sklearn.model_selection import cross_validate

## cv: 分割数(=k)
res = cross_validate(knn2, X_train1, y_train1, cv=5,
                     return_train_score=True)
display(pd.DataFrame(res))

#### 3.3.2 グリッドサーチ

**グリッドサーチ (Grid search)** とは指定したパラメータの全ての組み合わせに対して学習を行い、もっとも良い精度を示したパラメータを採用する手法です。

In [ ]:
# 単純なグリッドサーチ

best_score = 0
for nn in [1, 3, 5, 7, 9, 11]:
    knn = KNeighborsClassifier(n_neighbors=nn)
    knn.fit(X_train2, y_train2)
    score = knn.score(X_val2, y_val2)
    if score > best_score:
        best_score = score
        best_parameters = {'n_neighbors' : nn}

print("Best score: {:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

In [ ]:
# 交差検証を用いたグリッドサーチ

from sklearn.model_selection import GridSearchCV

knn_params = {'n_neighbors': [1, 3, 5, 7, 9, 11]}
knn_grid_search = GridSearchCV(
                    KNeighborsClassifier()
                    , knn_params
                    , cv=5
                    , return_train_score=True
                  )
knn_grid_search.fit(X_train2, y_train2)

print("Best cross-validation score: {:.2f}".format(knn_grid_search.best_score_))
print("Best parameters: {}".format(knn_grid_search.best_params_))

### 3.4 モデルの評価

In [ ]:
# 単純な生存率を確認

survive_rate = y_train2.sum()/len(y_train2)
print("Survive rate: {}".format(survive_rate))
print("Base line accuracy: {}".format(1 - survive_rate))

2クラス分類の評価結果を表現する方法の一つに **混同行列 (Confusion matrix)** があります。
<table>
	<tbody>
		<tr>
			<td colspan="2" rowspan="2"></td>
			<td colspan="2">モデルによる予測</td>
		</tr>
		<tr>
			<td>0</td>
			<td>1</td>
		</tr>
		<tr>
			<td rowspan="2">正解ラベル</td>
			<td>0
</td>
			<td>TN<br>(True Negative)</td>
			<td>FP<br>(False Positive)</td>
		</tr>
		<tr>
			<td>1</td>
			<td>FN<br>(False Negative)</td>
			<td>TP<br>(True Positive)</td>
		</tr>
	</tbody>
</table>

このように正解ラベルと予測に関してデータを振り分けると、各指標が次のように表せます。

**正解率 (Accuracy)** = $\frac{TP+TN}{TP+FN+FP+TN}$

**適合率 (Precision)** = $\frac{TP}{TP+FP}$

**再現率 (Recall)** = $\frac{TP}{TP+FN}$

**F-値 (F-measure)** = $\frac{2}{1/\mathrm{Precision}+1/\mathrm{Recall}}$ = $\frac{{2}\times{Precision}\times{Recall}}{{Precision+Recall}}$：適合率と再現率の調和平均

In [ ]:
# 交差検証を用いたグリッドサーチでできたモデルの予測結果を混同行列で表示

from sklearn.metrics import confusion_matrix

# 混同行列作成
pred_knn_grid_search = knn_grid_search.predict(X_test2)
knn_conf_matrix = confusion_matrix(y_test2, pred_knn_grid_search)

# データフレーム作成
class_names = ['died', 'survived']
df_knn_conf_matrix = pd.DataFrame(knn_conf_matrix,index=class_names,columns=class_names)

# グラフ作成
sns.heatmap(df_knn_conf_matrix, annot=True, cbar=None, cmap='Blues')
plt.title("Confusion Matrix")
plt.ylabel("True Class")
plt.xlabel("Predicted Class")
plt.show()

In [ ]:
# 各指標を出力

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("accuracy score knn: {:.2f}".format(accuracy_score(y_test2, pred_knn_grid_search)))
print("precision score knn: {:.2f}".format(precision_score(y_test2, pred_knn_grid_search)))
print("recall score knn: {:.2f}".format(recall_score(y_test2, pred_knn_grid_search)))
print("f1 score knn: {:.2f}".format(f1_score(y_test2, pred_knn_grid_search)))

In [ ]:
# レポートを出力

from sklearn.metrics import classification_report

### supportは個数を意味する
print(classification_report(y_test2, pred_knn_grid_search, target_names=['died', 'survived']))

### Scikit-learnにおける教師有り学習のレシピ

#### データを準備


#### 前処理

#### 訓練データとテストデータへの分割

```
sklearn.model_selection.train_test_split(, random_state=)
```

#### モデルの指定

```
from sklearn.<モジュール名> import <モデル名>
model = <モデル名>(ハイパーパラメータ)
```

#### モデルの学習

```
model.fit(X_train, y_train)
```

#### モデルの予測結果出力

```
y_pred = model.predict(X_test)
```